In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv
import math
import statistics
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from keras.layers.core import Dense, Activation, Dropout
import time  # helper libraries
import joblib

from dataset_window import Dataset


def convertCash(cash):
	cash = cash.replace('.', '')
	cash = cash.replace(',', '.')
	return float(cash)

def clearNanMM(moving_average, price):
        moving_average[0] = price[0]
        moving_average[1] = (price[0] + price[1])/2
        moving_average[2] = (price[0] + price[1] + price[2])/3
        return moving_average;


def create_dataset(dataset, look_back=1):
	dataX, dataY = [], [] 
	for i in range(len(dataset)-look_back):
		a = dataset[i:(i+look_back), :]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])

	return np.array(dataX), np.array(dataY)

#Carregando os dados
data = pd.read_csv("preços-2021.csv")
data['DATA'] = pd.to_datetime(data.DATA, format='%d/%m/%y')
data.set_index('DATA', inplace=True)
index = data.index.values
# print(index)
# data['PREÇO'] = data['PREÇO'].apply(convertCash)
data['PREÇO_DOLAR'] = data['PREÇO_DOLAR'].apply(convertCash)

import talib as ta

data['MEDIA_MOVEL'] = ta.EMA(data['PREÇO'], timeperiod=4)
data['MEDIA_MOVEL'] = clearNanMM(data['MEDIA_MOVEL'], data['PREÇO'])

<ipython-input-1-8b9812cb7d39>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moving_average[0] = price[0]
<ipython-input-1-8b9812cb7d39>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moving_average[1] = (price[0] + price[1])/2
<ipython-input-1-8b9812cb7d39>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moving_average[2] = (price[0] + price[1] + price[2])/3


In [2]:
scaler_price = joblib.load("scaler_price_file")
scaler_dollar = joblib.load("scaler_dollar_file") 
scaler_media_movel = joblib.load("scaler_media_movel_file") 

In [3]:
data['PREÇO'] = scaler_price.fit_transform(data['PREÇO'].to_numpy().reshape(-1, 1))
data['PREÇO_DOLAR'] = scaler_dollar.fit_transform(data['PREÇO_DOLAR'].to_numpy().reshape(-1, 1))
data['MEDIA_MOVEL'] = scaler_dollar.fit_transform(data['MEDIA_MOVEL'].to_numpy().reshape(-1, 1))

In [4]:
data_window = np.array([data.values])

print(data_window)

[[[1.         0.09031318 1.        ]
  [0.625      0.35251275 0.625     ]
  [0.375      1.         0.33333333]
  [0.         0.         0.        ]]]


In [5]:
model = load_model('price_model')

In [6]:
weekly_price_now = model.predict(data_window)

weekly_price_now = scaler_price.inverse_transform(weekly_price_now)

weekly_price_now

array([[24.045876]], dtype=float32)